In [1]:
import torch 
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from textpruner import TransformerPruner
from tqdm import tqdm

# Load model directly
tokenizer = AutoTokenizer.from_pretrained("hw2942/bert-base-chinese-finetuning-financial-news-sentiment-v2")
model = AutoModelForSequenceClassification.from_pretrained("hw2942/bert-base-chinese-finetuning-financial-news-sentiment-v2",output_attentions=True)

# load the dataset 
ds = load_dataset("hw2942/financial-news-sentiment")


d:\Python\lib\site-packages\transformers\modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cpu")
d:\Pytho

In [40]:
import torch

# Hook函数，用来替换注意力矩阵
def attention_hook(module, input, output, layer_idx, head_idx, scaled_attention):
    # 替换特定头的注意力矩阵
    print(output[0].size())
    output[layer_idx][:, head_idx, :, :] = scaled_attention

# 计算每个头的Self-Attention Attribution矩阵
def get_self_attention_attribution(model, input_data, tokenizer, device='cuda', m=10):
    model.to(device)
    inputs = tokenizer(input_data, return_tensors='pt').to(device)
    
    # 获取模型的原始输出，并启用 attentions 输出
    outputs = model(**inputs, output_attentions=True)
    
    # 获取 attentions（注意力矩阵）
    attentions = outputs.attentions  # 注意，这里是一个包含每层的注意力矩阵的元组
    num_layers = len(attentions)
    num_heads = attentions[0].size(1)  # 获取每层的注意力头数
    attribution_scores = []

    # 遍历每一层
    for layer_idx in range(num_layers):
        attention_layer = model.bert.encoder.layer[layer_idx].attention.self
        attention_weights = attentions[layer_idx]
        layer_attribution = []

        # 对每个头进行归因计算
        for head_idx in range(num_heads):
            # 初始化积分近似结果
            head_attribution = torch.zeros_like(attention_weights[:, head_idx, :, :]).to(device)

            # 计算归因：将注意力权重从0渐进到真实值
            for k in range(1, m + 1):
                # 权重比例
                alpha = k / m
                scaled_attention = alpha * attention_weights[:, head_idx, :, :].clone().detach().requires_grad_(True)

                # 使用forward hook修改注意力矩阵
                hook_handle = attention_layer.register_forward_hook(lambda module, input, outputs: 
                                                                    attention_hook(module, input, outputs, layer_idx, head_idx, scaled_attention))

                # 前向传播
                model.zero_grad()
                scaled_outputs = model(**inputs, output_attentions=True)
                loss = torch.norm(scaled_outputs.logits)

                # 反向传播，计算梯度
                loss.backward(retain_graph=True)
                grad_attention = scaled_attention.grad

                # 累加归因得分
                head_attribution += (grad_attention * (alpha / m))

                # 移除 hook
                hook_handle.remove()

            # 保存每个头的归因结果
            layer_attribution.append(head_attribution)
        attribution_scores.append(layer_attribution)

    return attribution_scores


In [6]:
inputs__ = tokenizer(ds['train'][0]['Title'],return_tensors='pt')

In [23]:
model.to('cpu')
model(**inputs__).attentions[0]

AttributeError: 'tuple' object has no attribute 'grad'

In [41]:
raw_attr_scores = get_self_attention_attribution(model,ds['train'][0]['Title'],tokenizer)
raw_attr_scores

torch.Size([1, 26, 768])


IndexError: too many indices for tensor of dimension 3

: 